In [ ]:
from dataset_utils import parse_dataset
from datasets import load_dataset, Dataset, DatasetDict

In [ ]:
orig_problems = parse_dataset("codegenning/F_taco-rl_Llama-3.1-8B-I_sols", "train")
orig_prob_data = load_dataset("codegenning/F_taco-rl_Llama-3.1-8B-I_sols")
dpo_split = load_dataset("codegenning/D_taco-rl_Llama-3.1-8B-I_sols")

In [ ]:
def filt(s: str):
    return s[10:-4]

splits = ["train", "test"]
new_dd = {}
for split in splits:
    all_idxs = []
    for chose in dpo_split[split]["chosen"]:
        query_str = filt(chose[2]["content"])
        good_idxs = []
        for i, prob in enumerate(orig_problems):
            if query_str in prob.solutions:
                good_idxs.append(i)
        assert len(good_idxs) == 1
        all_idxs.append(good_idxs[0])

    new_dd[split] = orig_prob_data["train"].select(all_idxs)

new_dd = DatasetDict(new_dd)

In [ ]:
new_dd.push_to_hub("codegenning/F_D_taco-rl_Llama-3.1-8B-I_sols", private=True)